<a href="https://colab.research.google.com/github/freentour/AIFFEL_quest/blob/main/Python_quest/Quest04/Quest04_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 계산기 프로그램 버전 2
"""
[주요 개선 사항]
- 허용되지 않은 연산자가 입력된 경우 사용할 예외 클래스로 MyOperatorError 클래스 추가 정의
- 0으로 나누는 경우, 계산 전용 함수인 calculator 함수를 호출하기 전에 미리 체크(결과적으로 calculator 함수가 호출될 때는 모든 사전 검사가 정상적으로 완료된 상태임)
- 수식을 입력받을 때 숫자와 연산자를 구분해서 입력받지 않고 수식 자체를 통으로 입력받아 처리
- 사용자 입력 내용이 유효한지 체크하는 부분을 모두 묶어서 하나의 try ~ except 블럭 내에서 처리
- 정수 외에 실수 형식도 처리할 수 있도록 수정
- 메인 프로그램을 클로저 함수로 만들어서 계산 횟수를 누적해서 카운트하고 계산 결과와 함께 출력
"""
from math import pow

# 연산자 입력 오류용 예외 클래스 정의 (Exception 클래스 상속)
class MyOperatorError(Exception):
  pass

# 계산 전용 함수 : 파라미터의 유효성은 모두 사전에 검사 완료된 상태에서 호출됨
def calculator(a, op, b):
  if op == "+":
    result = a+b
  elif op == "-":
    result = a-b
  elif op == "*":
    result = a*b
  elif op == "/":
    # calculator 함수 호출 전에 이미 0으로 나누는 상황을 점검했기 때문에 여기서는 그냥 계산만 하면 됨.
    result = a/b
  elif op == "**":
    result = pow(a,b)

  return result

# 메인 프로그램을 클로저 함수로 만들기
def outer():    # 외부 함수
  cal_count = 0   # 계산 횟수 누적용 변수(프리변수)

  def inner():   # 내부 함수
    nonlocal cal_count    # 프리변수를 내부 함수에서 수정하기 위한 설정

    # 메인 프로그램 시작
    print("계산기 프로그램을 시작합니다.")

    is_continue = True    # 프로그램 계속 여부를 결정하기 위한 플래그 변수

    while is_continue:   # 프로그램 계속 여부에 대한 반복문
      while True:   # 수식 입력에 대한 반복문
        print("-"*50)
        print("계산을 원하는 수식을 입력해 주세요. (예: 2 + 3)")
        print("- 주의 : 숫자와 연산자 사이에는 반드시 공백을 추가해 주세요.")
        print("- 가능한 연산자 종류 : +, -, *, /, **(제곱)")
        print("\n")

        exp = input("수식 : ")   # 수식 입력
        exp_list = exp.split(' ')   # 공백을 기준으로 분리해서 리스트로 저장

        # 일단 수식 구성요소는 3개까지만 가능. 그 이상은 안됨. (나중에 업그레이드 검토)
        if len(exp_list) < 3:
          print("[입력 오류] 수식이 너무 짧네요. 혹시 숫자와 연산자 사이에 공백을 빼먹지 않았는지 다시 한번 확인해 주세요")
          # print(exp_list)
          continue
        elif len(exp_list) > 3:
          print("[입력 오류] 수식이 너무 기네요. 아직은 두 개의 숫자에 대해서만 계산할 수 있어요. ㅠㅠ")
          # print(exp_list)
          continue

        # 수식 구성요소 분리
        first, operator, second = tuple(x for x in exp_list)    # 리스트 컴프리헨션 사용

        # 사용자 입력 내용 유효성 체크
        try:
          first = float(first)
          second = float(second)

          operator_list = ['+', '-', '*', '/', '**']
          if operator not in operator_list:
            raise MyOperatorError("[입력 오류] 허용되지 않은 연산자입니다. 수식을 다시 입력해 주세요.", 101, operator)

          if operator == '/' and second == 0:   # 0으로 나누려는 경우
            raise ZeroDivisionError("[입력 오류] 0으로 나눌 수 없습니다. 수식을 다시 입력해 주세요.", 102)
        except ValueError:
          print(f"[입력 오류] 숫자가 잘못 입력되었습니다. 수식을 다시 입력해 주세요. (Error Code: 103, 입력 내용: {exp})")
          continue
        except MyOperatorError as e:
          print(f"{e.args[0]} (Error Code: {e.args[1]}, 입력한 연산자: {e.args[2]})")
          continue
        except ZeroDivisionError as e:
          print(f"{e.args[0]} (Error Code: {e.args[1]}, 입력 내용: {exp})")
          continue
        else:   # 사용자 입력 내용에 오류가 없는 경우
          break   # [중요] 이거 빼먹으면 정상 입력인 경우에도 계속 반복해서 입력 요구함

      # 실제 계산은 calculator 함수에서 전담해서 처리
      result = calculator(first, operator, second)

      # 계산 횟수 누적용 변수 증가
      cal_count += 1

      # 계산 결과 출력
      print(f"결과 : {exp} = {result}")
      print(f"현재까지의 누적 계산 횟수 : {cal_count}")
      print("\n")

      while True:
        continue_chr = input("계속하시겠습니까? (Y/N) : ").lower()
        if continue_chr == 'y':
          break
        elif continue_chr == 'n':
          print("-"*50)
          print("계산기 프로그램을 종료합니다.")
          is_continue = False   # 제일 바깥 while문을 빠져나오기 위한 변수값 설정
          break
        else:
          print("'Y' 또는 'N'을 입력해 주세요.")
          continue

  return inner   # 내부 함수인 inner 함수 리턴

main = outer()   # main 변수에 outer로부터 리턴받은 inner 함수 바인딩
main()   # 바인딩된 inner 함수 호출(실행)


계산기 프로그램을 시작합니다.
--------------------------------------------------
계산을 원하는 수식을 입력해 주세요. (예: 2 + 3)
- 주의 : 숫자와 연산자 사이에는 반드시 공백을 추가해 주세요.
- 가능한 연산자 종류 : +, -, *, /, **(제곱)


수식 : 2 + 3
결과 : 2 + 3 = 5.0
현재까지의 누적 계산 횟수 : 1


계속하시겠습니까? (Y/N) : 2+4
'Y' 또는 'N'을 입력해 주세요.
계속하시겠습니까? (Y/N) : y
--------------------------------------------------
계산을 원하는 수식을 입력해 주세요. (예: 2 + 3)
- 주의 : 숫자와 연산자 사이에는 반드시 공백을 추가해 주세요.
- 가능한 연산자 종류 : +, -, *, /, **(제곱)


수식 : 2+4
[입력 오류] 수식이 너무 짧네요. 혹시 숫자와 연산자 사이에 공백을 빼먹지 않았는지 다시 한번 확인해 주세요
--------------------------------------------------
계산을 원하는 수식을 입력해 주세요. (예: 2 + 3)
- 주의 : 숫자와 연산자 사이에는 반드시 공백을 추가해 주세요.
- 가능한 연산자 종류 : +, -, *, /, **(제곱)


수식 : 2 +4
[입력 오류] 수식이 너무 짧네요. 혹시 숫자와 연산자 사이에 공백을 빼먹지 않았는지 다시 한번 확인해 주세요
--------------------------------------------------
계산을 원하는 수식을 입력해 주세요. (예: 2 + 3)
- 주의 : 숫자와 연산자 사이에는 반드시 공백을 추가해 주세요.
- 가능한 연산자 종류 : +, -, *, /, **(제곱)


수식 : 2 + 3 + 4
[입력 오류] 수식이 너무 기네요. 아직은 두 개의 숫자에 대해서만 계산할 수 있어요. ㅠㅠ
-------------------------------------